# Hierarchically Deep Convolutional Neural Network For Image Recognition

## Setup and Imports

**Import Packages**

In [1]:
import tensorflow as tf
from keras.datasets import cifar100
from tensorflow.keras.layers import Input, Dropout, Flatten, Dense, Activation, Lambda, Conv2D, MaxPool2D, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.utils import plot_model
import datetime, os
import time
import pandas as pd
import seaborn as sns


**Define Global Variables**

## Import and Preprocess Dataset

**Import Cifar100 Data Set**

In [2]:
#Load dataset
(X_c_train, y_c_train), (X_c_test, y_c_test) = cifar100.load_data(label_mode='coarse')
(X_f_train, y_f_train), (X_f_test, y_f_test) = cifar100.load_data(label_mode='fine')

#######################
# Clusters are obtained from:
# https://github.com/stephenyan1231/caffe-public/blob/hdcnn/examples/cifar100/hdcnn/python/cifar100_NIN_raw.ipynb
# Mapping fine -> cluster
mapping_fine_to_cluster = {0: 5,1: 2,2: 3,3: 6,4: 6,5: 0,6: 2,7: 2,8: 8,9: 1,10:1,11:3,12:8,13:8,14:2,15:6,16:1,17:8,18:2,19:6,20:0,21:3,22:1,23:7,24:2,25:0,26:2,27:6,28:1,29:6,30:6,31:6,32:6,33:4,34:6,35:3,36:6,37:8,38:6,39:1,40:1,41:8,42:6,43:6,44:2,45:2,46:3,47:4,48:8,49:7,50:6,51:5,52:4,53:5,54:5,55:6,56:4,57:5,58:8,59:4,60:7,61:1,62:5,63:6,64:6,65:6,66:6,67:6,68:7,69:1,70:5,71:7,72:6,73:2,74:6,75:6,76:1,77:2,78:2,79:2,80:6,81:8,82:2,83:5,84:0,85:8,86:1,87:0,88:6,89:8,90:8,91:2,92:5,93:6,94:0,95:6,96:4,97:6,98:3,99:2}
def map_fine_to_cluster(y_f):
  y_cluster = []
  for f in  y_f:
    k = f[0]
    c = np.array([mapping_fine_to_cluster[k]])
    y_cluster.append(c)
  return np.array(y_cluster)
  
y_c_train = map_fine_to_cluster(y_f_train)
y_c_test = map_fine_to_cluster(y_f_test)
########################


X_train = X_f_train
X_val = X_f_test[:5000]
X_test = X_f_test[5000:]

y_train = [y_c_train, y_f_train]
y_val = [y_c_test[:5000], y_f_test[:5000]]
y_test = [y_c_test[5000:], y_f_test[5000:]]

image_size = X_train[0].shape

num_classes_c = len(set([v[0] for v in y_c_train]))
num_classes_f = len(set([v[0] for v in y_f_train]))

# Encode matrix M
M = [[0 for x in range(num_classes_f)] for y in range(num_classes_c)] 
for (c, f) in zip(y_c_train, y_f_train):
  c = c[0]
  f = f[0]
  M[c][f] = 1

loss = keras.losses.SparseCategoricalCrossentropy()

## Single Classifier Training

**Constructing CNN**

In [38]:
from keras import optimizers
from keras.layers import Input, Conv2D, Dropout, MaxPooling2D, Flatten, Dense
from keras.models import Model

def get_model():
  in_layer = Input(shape=(32, 32, 3), dtype='float32', name='main_input')

  net = Conv2D(384, 3, strides=1, padding='same', activation='elu')(in_layer)
  net = MaxPooling2D((2, 2), padding='valid')(net)

  net = Conv2D(384, 1, strides=1, padding='same', activation='elu')(net)
  net = Conv2D(384, 2, strides=1, padding='same', activation='elu')(net)
  net = Conv2D(640, 2, strides=1, padding='same', activation='elu')(net)
  net = Conv2D(640, 2, strides=1, padding='same', activation='elu')(net)
  net = Dropout(.2)(net)
  net = MaxPooling2D((2, 2), padding='valid')(net)

  net = Conv2D(640, 1, strides=1, padding='same', activation='elu')(net)
  net = Conv2D(768, 2, strides=1, padding='same', activation='elu')(net)
  net = Conv2D(768, 2, strides=1, padding='same', activation='elu')(net)
  net = Conv2D(768, 2, strides=1, padding='same', activation='elu')(net)
  net = Dropout(.3)(net)
  net = MaxPooling2D((2, 2), padding='valid')(net)

  net = Conv2D(768, 1, strides=1, padding='same', activation='elu')(net)
  net = Conv2D(896, 2, strides=1, padding='same', activation='elu')(net)
  net = Conv2D(896, 2, strides=1, padding='same', activation='elu')(net)
  net = Dropout(.4)(net)
  net = MaxPooling2D((2, 2), padding='valid')(net)

  net = Conv2D(896, 3, strides=1, padding='same', activation='elu')(net)
  net = Conv2D(1024, 2, strides=1, padding='same', activation='elu')(net)
  net = Conv2D(1024, 2, strides=1, padding='same', activation='elu')(net)
  net = Dropout(.5)(net)
  net = MaxPooling2D((2, 2), padding='valid')(net)

  net = Conv2D(1024, 1, strides=1, padding='same', activation='elu')(net)
  net = Conv2D(1152, 2, strides=1, padding='same', activation='elu')(net)
  net = Dropout(.6)(net)
  net = MaxPooling2D((2, 2), padding='same')(net)

  net = Flatten()(net)
  net = Dense(1152, activation='elu')(net)
  net = Dense(num_classes_f, activation='softmax')(net)
  model = Model(inputs=in_layer, outputs=net)

  sgd_coarse = optimizers.SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
  model.compile(optimizer= sgd_coarse, loss=loss, metrics=['accuracy'])

  return model


**Compile Model**

In [40]:
model = get_model()
model.summary()


Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
main_input (InputLayer)      [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d_51 (Conv2D)           (None, 32, 32, 384)       10752     
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 16, 16, 384)       0         
_________________________________________________________________
conv2d_52 (Conv2D)           (None, 16, 16, 384)       147840    
_________________________________________________________________
conv2d_53 (Conv2D)           (None, 16, 16, 384)       590208    
_________________________________________________________________
conv2d_54 (Conv2D)           (None, 16, 16, 640)       983680    
_________________________________________________________________
conv2d_55 (Conv2D)           (None, 16, 16, 640)       1639

**Train Model**

In [43]:
batch = 128
model = get_model()
early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
history = model.fit(X_train, 
          np.array(y_train[1]),
          validation_data=(X_val, np.array(y_val[1])),
          batch_size=batch, 
          epochs=100,
          callbacks=[early_stopping_callback])


Epoch 1/100
391/391 [==============================] - 88s 218ms/step - loss: 4.5357 - accuracy: 0.0194 - val_loss: 3.9079 - val_accuracy: 0.0954
Epoch 2/100
391/391 [==============================] - 84s 216ms/step - loss: 3.8573 - accuracy: 0.0915 - val_loss: 3.4484 - val_accuracy: 0.1702
Epoch 3/100
391/391 [==============================] - 84s 216ms/step - loss: 3.3648 - accuracy: 0.1763 - val_loss: 3.0495 - val_accuracy: 0.2506
Epoch 4/100
391/391 [==============================] - 84s 216ms/step - loss: 2.9760 - accuracy: 0.2464 - val_loss: 2.9135 - val_accuracy: 0.2910
Epoch 5/100
391/391 [==============================] - 84s 216ms/step - loss: 2.6577 - accuracy: 0.3110 - val_loss: 2.7930 - val_accuracy: 0.3356
Epoch 6/100
391/391 [==============================] - 84s 216ms/step - loss: 2.3722 - accuracy: 0.3699 - val_loss: 2.5223 - val_accuracy: 0.3874
Epoch 7/100
391/391 [==============================] - 84s 216ms/step - loss: 2.1677 - accuracy: 0.4184 - val_loss: 2.4902 -

### Load Most Recent Model

In [45]:
for i in range(len(model.layers)):
    model.layers[i].trainable=False

In [52]:
model.layers[0].input

<KerasTensor: shape=(None, 32, 32, 3) dtype=float32 (created by layer 'main_input')>

## Fine-Tuning for Coarse Classifier

In [54]:
net = Conv2D(1024, 1, strides=1, padding='same', activation='elu')(model.layers[-8].output)
net = Conv2D(1152, 2, strides=1, padding='same', activation='elu')(net)
net = Dropout(.6)(net)
net = MaxPooling2D((2, 2), padding='same')(net)

net = Flatten()(net)
net = Dense(1152, activation='elu')(net)
out_coarse = Dense(num_classes_c, activation='softmax')(net)

model_c = Model(inputs=model.layers[0].input, outputs=out_coarse)
model_c.compile(optimizer= sgd_coarse, loss=loss, metrics=['accuracy'])

for i in range(len(model_c.layers)-1):
    model_c.layers[i].set_weights(model.layers[i].get_weights())


In [56]:
sgd_fine = optimizers.SGD(learning_rate=0.001, decay=1e-6, momentum=0.9, nesterov=True)
model_c.compile(optimizer=sgd_fine, loss=loss, metrics=['accuracy'])
history_c = model_c.fit(X_train, 
            np.array(y_train[0]), 
            batch_size=batch, 
            epochs=100, 
            validation_data=(X_val, np.array(y_val[0])), 
            callbacks=[early_stopping_callback])


Epoch 1/100
391/391 [==============================] - 34s 80ms/step - loss: 0.4788 - accuracy: 0.8359 - val_loss: 1.2484 - val_accuracy: 0.7656
Epoch 2/100
391/391 [==============================] - 31s 79ms/step - loss: 0.2823 - accuracy: 0.9021 - val_loss: 1.2151 - val_accuracy: 0.7662
Epoch 3/100
391/391 [==============================] - 31s 78ms/step - loss: 0.2569 - accuracy: 0.9107 - val_loss: 1.2315 - val_accuracy: 0.7714
Epoch 4/100
391/391 [==============================] - 31s 78ms/step - loss: 0.2569 - accuracy: 0.9119 - val_loss: 1.2622 - val_accuracy: 0.7660
Epoch 5/100
391/391 [==============================] - 31s 79ms/step - loss: 0.2515 - accuracy: 0.9134 - val_loss: 1.2921 - val_accuracy: 0.7706
Epoch 6/100
391/391 [==============================] - 31s 78ms/step - loss: 0.2448 - accuracy: 0.9151 - val_loss: 1.2640 - val_accuracy: 0.7752
Epoch 7/100
391/391 [==============================] - 30s 78ms/step - loss: 0.2403 - accuracy: 0.9156 - val_loss: 1.2983 - val_ac

## Fine-Tuning for Fine Classifiers

### Construct Fine Classifiers

In [57]:
def fine_model():
    net = Conv2D(1024, 1, strides=1, padding='same', activation='elu')(model.layers[-8].output)
    net = Conv2D(1152, 2, strides=1, padding='same', activation='elu')(net)
    net = Dropout(.6)(net)
    net = MaxPooling2D((2, 2), padding='same')(net)

    net = Flatten()(net)
    net = Dense(1152, activation='elu')(net)
    out_fine = Dense(num_classes_f, activation='softmax')(net)
    model_fine = Model(inputs=model.layers[0].input, outputs=out_fine)
    model_fine.compile(optimizer= sgd_coarse,
              loss=loss,
              metrics=['accuracy'])
    
    for i in range(len(model_fine.layers)-1):
        model_fine.layers[i].set_weights(model.layers[i].get_weights())
    return model_fine

In [60]:
fine_models = {'models' : [{} for i in range(num_classes_c)], 'yhf' : [{} for i in range(num_classes_c)]}
for i in range(num_classes_c):
    model_i = fine_model()
    fine_models['models'][i] = model_i

### Train Fine Classifiers on Respective Data

In [61]:
def get_error(y,yh):
    # Threshold 
    yht = np.zeros(np.shape(yh))
    yht[np.arange(len(yh)), yh.argmax(1)] = 1
    # Evaluate Error
    error = np.count_nonzero(np.count_nonzero(y-yht,1))/len(y)
    return error

In [ ]:
for i in range(coarse_categories):
    index= 0
    step = 5
    stop = 5
    
    # Get all training data for the coarse category
    ix = np.where([(y_train[:,j]==1) for j in [k for k, e in enumerate(fine2coarse[:,i]) if e != 0]])[1]
    x_tix = x_train[ix]
    y_tix = y_train[ix]
    
    # Get all validation data for the coarse category
    ix_v = np.where([(y_val[:,j]==1) for j in [k for k, e in enumerate(fine2coarse[:,i]) if e != 0]])[1]
    x_vix = x_val[ix_v]
    y_vix = y_val[ix_v]
    
    while index < stop:
        fine_models['models'][i].fit(x_tix, y_tix, batch_size=batch, initial_epoch=index, epochs=index+step, validation_data=(x_vix, y_vix))
        index += step
    
    fine_models['models'][i].compile(optimizer=sgd_fine, loss='categorical_crossentropy', metrics=['accuracy'])
    stop = 10

    while index < stop:
        fine_models['models'][i].fit(x_tix, y_tix, batch_size=batch, initial_epoch=index, epochs=index+step, validation_data=(x_vix, y_vix))
        index += step
        
    yh_f = fine_models['models'][i].predict(x_val[ix_v], batch_size=batch)
    print('Fine Classifier '+str(i)+' Error: '+str(get_error(y_val[ix_v],yh_f))) 

## Probabilistic Averaging

In [ ]:
def eval_hdcnn(X, y):
    yh = np.zeros(np.shape(y))
    
    yh_s = model.predict(X, batch_size=batch)
    
    print('Single Classifier Error: '+str(get_error(y,yh_s)))
    
    yh_c = model_c.predict(X, batch_size=batch)
    y_c = np.dot(y,fine2coarse)
    
    print('Coarse Classifier Error: '+str(get_error(y_c,yh_c)))

    for i in range(coarse_categories):
        if i%5 == 0:
            print("Evaluating Fine Classifier: ", str(i))
        #fine_models['yhf'][i] = fine_models['models'][i].predict(X, batch_size=batch)
        yh += np.multiply(yh_c[:,i].reshape((len(y)),1), fine_models['yhf'][i])
    
    print('Overall Error: '+str(get_error(y,yh)))
    return yh

In [ ]:
yh = eval_hdcnn(x_val,y_val)